<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv">link</a>.

**NOTE**:

For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01"><img src="https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [1]:
%load_ext sql

In [2]:
dsn_hostname = "9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "hdf12263"
dsn_pwd = "uCbxJLt34fe22eqQ"
dsn_database = "bludb"
dsn_port = "32459"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

connection_string = 'ibm_db_sa://{0}:{1}@{2}:{3}/{4}?security={5}'.format(dsn_uid, dsn_pwd, dsn_hostname, dsn_port, dsn_database, dsn_security)

In [3]:
print(connection_string)

ibm_db_sa://hdf12263:uCbxJLt34fe22eqQ@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL


In [4]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://hdf12263:uCbxJLt34fe22eqQ@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [5]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT * FROM SYSCAT.TABLES WHERE TABNAME = 'SCHOOLS';

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
HDF12263,SCHOOLS,HDF12263,U,T,N,None,None,None,None,2024-07-22 18:08:39.313838,2024-07-22 18:08:39.313838,2024-07-22 18:08:39.313838,2024-07-22 18:09:21.764445,78,4,4400,566,15,0,16,-1,-1,-1,0,HDF12263SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,843,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2024-07-22 18:08:39.313838,0,,None,None,N,HDF12263,,,,0001-01-01,,N,R,N,-1.0,None


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [6]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql SELECT COUNT(*) FROM SYSCAT.COLUMNS WHERE TABNAME = 'SCHOOLS';

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


1
78


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [7]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT COLNAME, TYPENAME, LENGTH FROM SYSCAT.COLUMNS WHERE TABNAME = 'SCHOOLS';

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


colname,typename,length
STUDENTS_TAKING__ALGEBRA__,VARCHAR,4
STUDENTS_PASSING__ALGEBRA__,VARCHAR,4
9th Grade EXPLORE (2009),VARCHAR,4
9th Grade EXPLORE (2010),VARCHAR,4
10th Grade PLAN (2009),VARCHAR,4
10th Grade PLAN (2010),VARCHAR,4
NET_CHANGE_EXPLORE_AND_PLAN,VARCHAR,3
11th Grade Average ACT (2011),VARCHAR,4
NET_CHANGE_PLAN_AND_ACT,VARCHAR,3
COLLEGE_ELIGIBILITY__,VARCHAR,4


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [8]:
%sql SELECT COUNT(*) FROM SCHOOLS WHERE "ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL" = 'ES';

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


1
462


### Problem 2

##### What is the highest Safety Score?


In [9]:
%sql SELECT MAX(SAFETY_SCORE) AS MAX_SAFETY_SCORE FROM schools;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


max_safety_score
99.0


### Problem 3

##### Which schools have highest Safety Score?


In [10]:
%sql SELECT NAME_OF_SCHOOL, SAFETY_SCORE FROM SCHOOLS WHERE SAFETY_SCORE = 99

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99.0
Alexander Graham Bell Elementary School,99.0
Annie Keller Elementary Gifted Magnet School,99.0
Augustus H Burley Elementary School,99.0
Edgar Allan Poe Elementary Classical School,99.0
Edgebrook Elementary School,99.0
Ellen Mitchell Elementary School,99.0
James E McDade Elementary Classical School,99.0
James G Blaine Elementary School,99.0
LaSalle Elementary Language Academy,99.0


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [11]:
%sql SELECT NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE from SCHOOLS \
    ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC NULLS LAST LIMIT 10;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,average_student_attendance
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Rachel Carson Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%
John H Vanderpoel Elementary Magnet School,97.20%


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [12]:
%sql SELECT NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE \
     FROM SCHOOLS \
     ORDER BY AVERAGE_STUDENT_ATTENDANCE \
     LIMIT 5;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [13]:
%sql SELECT NAME_OF_SCHOOL, REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '') \
     FROM SCHOOLS \
     ORDER BY AVERAGE_STUDENT_ATTENDANCE \
     LIMIT 5;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,2
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00
Orr Academy High School,66.30


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [14]:
%sql SELECT NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE \
     FROM SCHOOLS \
     WHERE CAST (REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '') AS DOUBLE ) < 70 \
     ORDER BY AVERAGE_STUDENT_ATTENDANCE;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


### Problem 8

##### Get the total College Enrollment for each Community Area


In [15]:
%sql SELECT COMMUNITY_AREA_NAME, sum(COLLEGE_ENROLLMENT_RATE__) AS TOTAL_ENROLLMENT \
   FROM SCHOOLS \
   GROUP BY COMMUNITY_AREA_NAME;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


community_area_name,total_enrollment
ALBANY PARK,42.9
ARCHER HEIGHTS,61.3
ARMOUR SQUARE,None
ASHBURN,50.0
AUBURN GRESHAM,None
AUSTIN,151.2
AVALON PARK,54.0
AVONDALE,None
BELMONT CRAGIN,105.0
BEVERLY,None


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [16]:
%sql SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS TOTAL_ENROLLMENT \
   FROM SCHOOLS \
   GROUP BY COMMUNITY_AREA_NAME \
   ORDER BY TOTAL_ENROLLMENT ASC \
   FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


community_area_name,total_enrollment
OAKLAND,140
FULLER PARK,531
BURNSIDE,549
OHARE,786
LOOP,871


### Problem 10

##### List 5 schools with lowest safety score.


In [17]:
%sql SELECT NAME_OF_SCHOOL, SAFETY_SCORE \
FROM SCHOOLS \
ORDER BY SAFETY_SCORE \
LIMIT 5

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


name_of_school,safety_score
Edmond Burke Elementary School,1.0
Luke O'Toole Elementary School,5.0
George W Tilton Elementary School,6.0
Foster Park Elementary School,11.0
Emil G Hirsch Metropolitan High School,13.0


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [18]:
%%sql 
SELECT HARDSHIP_INDEX
   FROM CHICAGO_SOCIOECONOMIC_DATA CD, schools CPS 
   WHERE CD.CA = CPS.COMMUNITY_AREA_NUMBER
      AND COLLEGE_ENROLLMENT = 4368

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


hardship_index
6


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [19]:
%sql SELECT CA, COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CHICAGO_SOCIOECONOMIC_DATA \
   WHERE CA IN \
   ( SELECT COMMUNITY_AREA_NUMBER FROM SCHOOLS ORDER BY COLLEGE_ENROLLMENT DESC LIMIT 1 );

 * ibm_db_sa://hdf12263:***@9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32459/bludb?security=SSL
Done.


ca,community_area_name,hardship_index
5,North Center,6


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2021-07-09        | 2.4     | Malika     | Updated connection string  |
| 2021-05-19        | 2.3     | Lakshmi Holla |Updated question     |
| 2021-04-20        | 2.2     | Malika     | Added the libraries                |
| 2020-11-27        | 2.1     | Sannareddy Ramesh    | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
